In [1]:
import nltk
import numpy as np
from nltk.corpus import stopwords, wordnet
from nltk.stem import PorterStemmer, SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer, StandardScaler
from sklearn.svm import SVC

from joblib import dump
from src.dataset_utils import MotleyFoolDataset

In [2]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger') 

[nltk_data] Downloading package stopwords to
[nltk_data]     /export/livia/home/vision/Ddjomby/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /export/livia/home/vision/Ddjomby/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /export/livia/home/vision/Ddjomby/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /export/livia/home/vision/Ddjomby/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

# Load and prepare dataset

Should be `(1141, 3821, 33174)`

In [3]:
dataset = MotleyFoolDataset('TMF_dataset_annotated.zip')
test_set = dataset['Q1 2023':'Q4 2023']
val_set = dataset['Q1 2022':'Q4 2022']
train_set = dataset[:'Q4 2021']

len(test_set), len(val_set), len(train_set)

(1141, 3821, 33174)

In [4]:
X_train_val = [instance['content'] for instance in dataset[:'Q4 2022']]
y_train_val = [instance['direction'] == 'UP' for instance in dataset[:'Q4 2022']]

X_test = [instance['content'] for instance in test_set]
y_test = [instance['direction'] == 'UP' for instance in test_set]

# Utilities for the grid search

In [5]:
stopwords_set = set(stopwords.words('english'))  # set for membership check optim
lemmatizer = WordNetLemmatizer()
porter = PorterStemmer()
snowball = SnowballStemmer("english")

### Adapted from: https://stackoverflow.com/questions/15586721/wordnet-lemmatization-and-pos-tagging-in-python
treebank_mapping = {
    'J': wordnet.ADJ,
    'V': wordnet.VERB,
    'N': wordnet.NOUN,
    'R': wordnet.ADV
}
treebank_default = wordnet.NOUN

def get_wordnet_pos(treebank_tag):
    return treebank_mapping.get(treebank_tag[0], treebank_default)

        
# Define custom NLTK tokenizers
def porter_tokenizer(text):
    return [porter.stem(word) for word in word_tokenize(text) if word not in stopwords_set]

def snowball_tokenizer(text):
    return [snowball.stem(word) for word in word_tokenize(text) if word not in stopwords_set]

def wordnet_lemmatizer_tokenizer(text):
    text = word_tokenize(text)
    tags = nltk.pos_tag(text)
    return [
        lemmatizer.lemmatize(word, get_wordnet_pos(tag))
        for word, tag in tags
        if word not in stopwords_set
    ]

In [6]:
vect_parameter_grid = {
    "vect__tokenizer": (
        wordnet_lemmatizer_tokenizer,
        porter_tokenizer,
        snowball_tokenizer,
    ),
    "vect__max_df": (0.2, 0.5, 0.8, 1.0),
    "vect__ngram_range": ((1, 1), (1, 2), (2, 2)),  # unigrams or bigrams
    "vect__norm": ("l1", "l2", None),
}

vectorizer_factory = lambda: ("vect", TfidfVectorizer(
    stop_words=None,
    token_pattern=None, # will not be used since 'tokenizer' is not None
))

def do_grid_search(parameter_grid, classifier=None, pipeline=None):
  parameter_grid |= vect_parameter_grid

  if pipeline is None:
    pipeline = Pipeline([
      vectorizer_factory(),
      ("clf", classifier),
    ])

  grid_search = GridSearchCV(
      estimator=pipeline,
      scoring='accuracy',
      param_grid=parameter_grid,
      n_jobs=-1,
      cv=[  # Remove cross validation and use our pre-made split
          # An iterable yielding (train, test) splits as arrays of indices.
          (np.arange(*dataset.range(q_stop='Q4 2021')),
           np.arange(*dataset.range('Q1 2022', 'Q4 2022')))
      ],
      verbose=3,
  )

  grid_search.fit(X_train_val, y_train_val)

  print('Best parameters:')
  best_parameters = grid_search.best_estimator_.get_params()
  for param_name in sorted(parameter_grid.keys()):
      print(f"{param_name}: {best_parameters[param_name]}")


  y_pred = grid_search.predict(X_test)
  ConfusionMatrixDisplay.from_predictions(y_test, y_pred)

  print('Accuracy score:', accuracy_score(y_test, y_pred))
  print(classification_report(y_test, y_pred)) 

  return grid_search

# Models

## Naive Bayes

In [ ]:
nb_gs = do_grid_search(
    {"clf__alpha": np.logspace(-6, 6, 13)},
    MultinomialNB()
)

# dump(nb_gs.best_estimator_, 'nb_gs.joblib') 

Fitting 1 folds for each of 1404 candidates, totalling 1404 fits


Traceback (most recent call last):
  File "<frozen runpy>", line 189, in _run_module_as_main
  File "<frozen runpy>", line 112, in _get_module_details
  File "/export/livia/home/vision/Ddjomby/anaconda3/envs/comp550/lib/python3.11/site-packages/joblib/__init__.py", line 129, in <module>
    from .parallel import Parallel
  File "/export/livia/home/vision/Ddjomby/anaconda3/envs/comp550/lib/python3.11/site-packages/joblib/parallel.py", line 31, in <module>
    from ._parallel_backends import (FallbackToBackend, MultiprocessingBackend,
  File "/export/livia/home/vision/Ddjomby/anaconda3/envs/comp550/lib/python3.11/site-packages/joblib/_parallel_backends.py", line 16, in <module>
    from .pool import MemmappingPool
  File "/export/livia/home/vision/Ddjomby/anaconda3/envs/comp550/lib/python3.11/site-packages/joblib/pool.py", line 31, in <module>
    from ._memmapping_reducer import get_memmapping_reducers
  File "/export/livia/home/vision/Ddjomby/anaconda3/envs/comp550/lib/python3.11/site-

## Logistic Regression

In [ ]:
lf_gs = do_grid_search(
    {
        "clf__penalty": ('l1', 'l2', 'elasticnet', None),
        "clf__C": np.logspace(-6, 6, 13),
    },
    LogisticRegression()
)

dump(nb_gs.best_estimator_, 'nb_gs.joblib') 

## Support Vector Machine

### Linear SVC

In [ ]:
svc_gs = do_grid_search(
    {"clf__C": np.logspace(-6, 6, 13)},
    pipeline=Pipeline([
      vectorizer_factory(),
      ("lsa", TruncatedSVD(n_components=100)),
      ("norm", StandardScaler()),
      ("clf", SVC(kernel='linear')) # So that it still is a linear classifier
    ])
)

dump(nb_gs.best_estimator_, 'nb_gs.joblib') 